# ITPS Flight Test Analysis Tool - v2.1
<p>Use this tool on FIREFOX and CHROME only </p>

<p>Safari and Edge have a few compatibility issues</p>

Currently the .csv files generated by the following devices are compatible:

- IADS
- PDAS
- iLevil
- Garmin G3X

In [ ]:
# Data File Selection
from FTAT_imports import *

# make display 100% of the screen
display(HTML("<style>.container { width:100% !important; }</style>"))

# set this variable to True to bypass file import and generate fake data
unit_test = False

if not unit_test:
    f = FileBrowser()
    display(f.widget())  #
    #   <interact with widget, select a path>

In [ ]:
# Data Import

if unit_test:
    # creating fake data just to test
    rng = pd.date_range('27/10/2018 13:00:00', periods=5000, freq='50ms')
    raw_data = pd.DataFrame(data=rng, columns=['Time'])
    raw_data.set_index(['Time'], inplace=True)
    raw_data['counter'] = np.arange(len(raw_data))
    
    raw_data['fake_angle'] = raw_data.apply(lambda x: (raw_data['counter']/10)%(2*np.pi))
    raw_data['all_zeroes'] = np.zeros(len(raw_data.index.values))
    raw_data['sin0'] = raw_data['counter'] #I have no idea why, but if I do not first create 'sine1' and then calculate with lambda, it throwns me an error
    raw_data['sin0'] = raw_data.apply(lambda x: np.sin(raw_data['fake_angle']))
    raw_data['cos0'] = raw_data['counter']
    raw_data['cos0'] = raw_data.apply(lambda x: np.cos(raw_data['fake_angle']))
    for i in range(5):
        sine_label = 'sin' + str(i+1)
        cosine_label = 'cos' + str(i+1)
        raw_data[sine_label] = raw_data['sin0']
        raw_data[cosine_label] = raw_data['cos0'] 
else:
    # select file and read from disk
    print('Preparing Graph Parameters ... may take a while ... patience is a virtue.')
    print('Reading data from {} ...'.format(f.path))
    #raw_data=pd.read_csv(filepath, encoding='latin1', low_memory=False) #
    
    filename = f.path
    filetype = 'IADS'
    with open(filename) as fp:
        first_line = fp.readline()
    if 'iLevil' in first_line:
        print('iLevil file detected')
        raw_data=pd.read_csv(filename, encoding='latin1', low_memory=False, skiprows=5)
        raw_data['Time'] = raw_data['UTC_TIME'] + "." + raw_data['TIMER(ms)'].apply(str)
        filetype = 'ilevil'
        print('File input finished.')
        raw_data.drop(['LEGEND','DATE', 'UTC_TIME'],axis=1, inplace=True)
        
    elif 'Analog 1' in first_line:
        print('PDAS file detected')
        raw_data=pd.read_csv(filename, encoding='latin1', low_memory=False)
        #raw_data['Time'] = raw_data['Time (s)'].apply(get_time)
        raw_data['delta_seconds'] = pd.to_timedelta(raw_data['Time (s)'] - raw_data['Time (s)'][0], unit='s')
        raw_data['Time'] = pd.to_datetime(weeksecondstoutc(float(raw_data['GNSS Week'][0]),float(raw_data['GNSS TOW (s)'][0]), 0,18)) + raw_data['delta_seconds']
        raw_data.drop(['delta_seconds'], axis=1, inplace=True)
        filetype = 'PDAS'
        print('File input finished.')
        
    elif '#airframe_info' in first_line:
        print('G3X file detected')
        raw_data=pd.read_csv(filename, encoding='latin1', low_memory=False, skiprows=[0,2,3,4,5,6,7,8,9,10]) #this is necessary to clean empty rows at the start of the file
        #print(raw_data.columns)
        #print(raw_data.head())
        raw_data['Time'] = raw_data.apply(G3Xweeksecondstoutc, args= (-18,), axis=1)
        raw_data.drop(['Date (yyyy-mm-dd)', 'Time (hh:mm:ss)', 'UTC Time (hh:mm:ss)', 'UTC Offset (hh:mm)', ], axis=1, inplace=True)
        filetype = 'G3X'
        print('File input finished.')

        
    elif len(first_line) == 1:
        print('X-Plane file detected')
        raw_data=pd.read_csv(filename, encoding='latin1', low_memory=False, skiprows=1, delimiter='|')
        #raw_data['Time'] = raw_data['Time (s)'].apply(get_time)
        #raw_data['delta_seconds'] = pd.to_timedelta(raw_data['   _real,_time'] - raw_data['   _real,_time'][0], unit='s')
        raw_data['Time'] = raw_data['   _real,_time '].apply(get_time)
        filetype = 'X-Plane'
        print('File input finished.')
        
    else:
        print('Reading IADS file')
        raw_data=pd.read_csv(filename, encoding='latin1', low_memory=False)
        print('File input finished.')
        dirty_file = False
        if raw_data['Time'][10].count(':') > 2:
            dirty_file = True

        if dirty_file == True:
            ## FOR DIRTY DATA
            print('Dirty file detected ... cleaning up the data...')  #DIRTY DATA
            raw_data.fillna(value=0, inplace=True)  #DIRTY DATA
            raw_data['Time'] = (raw_data['Time'].str.slice_replace(0,4,''))  #DIRTY DATA

    raw_data['Time'] = pd.to_datetime(raw_data['Time'])   #CLEAN DATA
    raw_data = raw_data.set_index(['Time'])   #CLEAN DATA

print('Done.')
print('Imported data from: {} to: {}, with {} records'.format(raw_data.index[0], raw_data.index[-1], len(raw_data.index) ))


In [ ]:
# Main Program

# Let's subsample to speed up the graphing process
desired_sub_sample_rate = 5 #Hz
counter = 0
start_sample = raw_data.index[0]
next_sample = start_sample
while (next_sample - start_sample <= np.timedelta64(1, 's')):
    counter = counter + 1
    next_sample = raw_data.index[counter]
if desired_sub_sample_rate <= counter:
    sub_sample_rate = int(counter / desired_sub_sample_rate)
else:
    sub_sample_rate = int(counter)
    desired_sub_sample_rate = counter
    
sub_sampled_data = raw_data[::sub_sample_rate]

print()
print('Original data at {} Hz'.format((counter-1)))
print('Data subsampled to {} Hz to speed up graphing. Reduced to {} records'.format(desired_sub_sample_rate, len(sub_sampled_data.index)))
print('Done.')

time_slices_db = {} #this is the time slices data base, TP#, Description, Start, Stop (indexes)
selected_slice = [] #this is a global variable to store the currently selected slice that will be stored in the time_slices_db
map_groups = {} #ths is for grouping parameters in the map

######################### Slice Map and Definitions ##################################################
for p in sub_sampled_data.columns:
    map_groups[p] = 'FTI'
if not unit_test:
    map_groups = get_param_group(f.path, filetype, raw_data, map_groups)
slicemap = ParameterMap(sub_sampled_data, 'Parameter Map', map_groups)

######################### Slice Graph Definitions ####################################################

current_plot_data = sub_sampled_data

slice_plot = LinePlotBrush(current_plot_data.index.values, current_plot_data.index.values)
detail_plot = LinePlot(current_plot_data.index.values, current_plot_data.index.values)
detail_plot_stats = widgets.HTML(value="Empty <b>Empty</b>", placeholder='Stats', description='Stats')
detail_plot_stats.value = 'Empty'

slicebox = sliceSelectDialog(current_plot_data)

######################### Parameter Selection and Detail Plot Updater #########################################


def detail_plot_update_brush(*args):
    lib_detail_plot_update_brush(*args, cpt=current_plot_data, selsl=selected_slice, detplt=detail_plot, slcplt=slice_plot, \
                                 slcbx=slicebox, dtstats=detail_plot_stats)

slice_plot.brushintsel.observe(detail_plot_update_brush, 'selected')


def selected_index_changed(change):
    '''
    This callback treats the parameter selection/deselection on the map
    Depending on the active tab, it will also callback the appropriate graph updater
    '''
    global current_plot_data, sub_sampled_data
    selected_tab = FTAT_tabs.selected_index
    if (selected_tab == 0 or selected_tab ==2):
        #this is for time slice or analysis tabs
        #define the current plot data per selected parameters
        if len(slicemap.map.selected) != 0:
            selected_parameters_plot_data = np.transpose(sub_sampled_data[slicemap.map.selected].values)
            current_plot_data = sub_sampled_data[slicemap.map.selected]
    if selected_tab == 0:
        #this is the time slice tab
        if (len(slicemap.map.selected) != 0) and (True not in pd.isnull(selected_parameters_plot_data)):
            safe_set_sc_n_dt(plt=slice_plot, pltdt=selected_parameters_plot_data)
            safe_set_sc_n_dt(plt=detail_plot, pltdt=selected_parameters_plot_data)
            slice_plot.fig.title = str(slicemap.map.selected)
            detail_plot.fig.title = str(slicemap.map.selected) + ' - Zoomed'
    elif selected_tab == 1:
        #this is the save to disk tab
        pass
    elif selected_tab == 2:
        #this is the analysis tab
        if len(slicemap.map.selected) != 0:
            safe_set_sc_n_dt(plt=analysis_plot, pltdt=selected_parameters_plot_data)

            analysis_plot.y_data = selected_parameters_plot_data  ##
            analysis_plot.fig.title = str(slicemap.map.selected)
            update_analysis_plot(current_plot_data, slicemap.map.selected, analysis_plot, poly_order, \
                                 analysis_plot_zoom_slider)
    elif selected_tab == 3:
        #strip chart tab
        pass
    elif selected_tab == 4:
        #filter tab
        pass
    else:
        pass

# link selection handler to parameter map
slicemap.map.observe(selected_index_changed, 'selected')

############################### Plot Button Logic #########################
plot_button = widgets.Button(description='Plot Slice Below')

def on_plot_button_clicked(b):
    lib_on_plot_button_clicked(b, cpt=current_plot_data, detplt=detail_plot, slcbx=slicebox, dtstats=detail_plot_stats)
    
plot_button.on_click(on_plot_button_clicked)

########################## Save Slice Button and Logic ##############################

# time slices are saved to a dictionary for later analysis
TP_number_box = widgets.FloatText(value=0.0, description='TP#:', disabled=False, layout=Layout(width='130px'))

TP_desc_box = widgets.Text(value='', placeholder='Type something', description='TP Title:', disabled=False)
TS_message = Label()


################## Data Slice Selector Instance ####################
data_slice_selec = DataSliceSelect(time_slices_db)

################## Add/Remove Time Slices to db Logic #######################

def update_TP_saved_display(time_slices_db):
    display_text = 'TP#  Description \n'
    for key in time_slices_db:
        display_text = display_text  + str(key) + '  ' + time_slices_db[key][0] + '\n'
    return display_text

        
def on_saveTS_button_clicked(b):
    lib_on_saveTS_button_clicked(b, cpt=current_plot_data, detplt=detail_plot, slcplt=slice_plot, \
                                tsdb=time_slices_db, tpnb=TP_number_box, tsmsg=TS_message, \
                                tpdescb=TP_desc_box, tpsavdd=TP_saved_dd, tsdbrb=time_slices_db_radio)
        

def on_delTS_button_clicked(b):
    lib_on_delTS_button_clicked(b, tsdb=time_slices_db, tsmsg=TS_message, \
                                tpsavdd=TP_saved_dd, tsdbrb=time_slices_db_radio)

TP_saved_dd = widgets.Dropdown(options=time_slices_db, description='TP#:', disabled=False,)

    
saveTS_button = widgets.Button(description='Save TSlice')
saveTS_button.on_click(on_saveTS_button_clicked)

delTS_button = widgets.Button(description='Delete TSlice')
delTS_button.on_click(on_delTS_button_clicked)

################## TS Radio Buttons ###################################################

time_slices_db_radio = widgets.RadioButtons(options=[], description='Slices:', disabled=False)

def on_click_time_slices_db_radio(change):
    lib_on_click_time_slices_db_radio(change, cpt=current_plot_data, anplt=analysis_plot, \
                                     tsdb=time_slices_db, pord=poly_order, \
                                     slcmap=slicemap, zsld=analysis_plot_zoom_slider)

time_slices_db_radio.observe(on_click_time_slices_db_radio, 'value')

################## Save Slices to Disk Logic ################################################
        
def on_save_slices_button_clicked(b):
    lib_save_slices(tsdb=time_slices_db, svall=save_all_parameters, rawdt=raw_data, \
                   slcmap=slicemap, svfb=save_feedback)     

save_instructions = widgets.Label(value='Select parameters to save or select checkbox below to save all', disable=False)

save_feedback = widgets.Label(value='', disable=False)

save_all_parameters = widgets.Checkbox(value=False, description='save ALL PARAMETERS', disabled=False)

save_slices_button = widgets.Button(description='Save Slices to Disk')
save_slices_button.on_click(on_save_slices_button_clicked)


################# Analysis Plot Definition #####################





analysis_plot = AnalysisPlot(current_plot_data.index, current_plot_data.index.values)
poly_order = 1

analysis_poly_order_dd = widgets.Dropdown(options=[1,2,3,4], description='Poly Deg', value=1, disabled=False)


def on_change_analysis_poly_order_dd(change):
    global poly_order, current_plot_data
    if change['type'] == 'change' and change['name'] == 'value' and len(slicemap.map.selected) != 0:
        poly_order = change['new']
        update_analysis_plot(current_plot_data, slicemap.map.selected, analysis_plot, poly_order, analysis_plot_zoom_slider)
            
analysis_poly_order_dd.observe(on_change_analysis_poly_order_dd)

analysis_plot_zoom_slider = SimpleZoomSlider(analysis_plot)

def on_analysis_zoom_slider_change(change):
    update_analysis_plot(current_plot_data, slicemap.map.selected, analysis_plot, poly_order, analysis_plot_zoom_slider)


analysis_plot_zoom_slider.LH_slider.observe(on_analysis_zoom_slider_change, names='value')
analysis_plot_zoom_slider.RH_slider.observe(on_analysis_zoom_slider_change, names='value')


def on_slice_trim_button_clicked(b):
    slice_trim(zsld=analysis_plot_zoom_slider, anplt=analysis_plot, tsdb=time_slices_db, tsdbr=time_slices_db_radio, \
              cpt=current_plot_data, pord=poly_order, slcmap=slicemap)

    
slice_trim_button = widgets.Button(description='Trim Current Slice')
slice_trim_button.on_click(on_slice_trim_button_clicked)

################# Strip Charts Definition #####################

strip_chart_button = widgets.Button(description='Plot Strip Chart Below')

strip_chart = StripChart(raw_data, slicemap, time_slices_db, time_slices_db_radio)

################ Filter Definitions ##########################

filter_label = widgets.Label('Low Pass Filter Parameters:')

filter_order = widgets.BoundedIntText(
            value=2,
            min=1,
            max=8,
            step=1,
            description='Filter Order:',
            disabled=False,
            layout=Layout(width='140px')
        )

filter_Wn = widgets.BoundedFloatText(
            value=0.01,
            min=0.001,
            max=0.999,
            step=0.01,
            description='3db kink pt:',
            disabled=False,
            layout=Layout(width='140px')
        )

apply_filter_button = widgets.Button(description='Apply Filter')

    
def on_apply_filter_button_clicked(b):
    global sub_sampled_data
    filter_data = raw_data[slicemap.map.selected].columns
    N = filter_order.value #filter order
    Wn = filter_Wn.value #3db cutoff kink point
    B, A = signal.butter(N, Wn, output='ba')
    
    for i in filter_data:
        if '_jfilt' not in i:
            raw_data[i+'_jfilt'] = signal.filtfilt(B, A, raw_data[i])
    
    for p in raw_data.columns:
        if not map_groups.get(p):
            map_groups[p] = 'filter'

    sub_sampled_data = raw_data[::sub_sample_rate]
    slicemap.update_map(sub_sampled_data, map_groups)


apply_filter_button.on_click(on_apply_filter_button_clicked)
#####################################################################################################
# MAIN TABS DEFINITIONS
#####################################################################################################

tab_contents = ['Slice', 'Save', 'Analysis', 'StripChart', 'Filter']
text_children = [widgets.Text(description=name) for name in tab_contents]

slice_tab_children = VBox([slice_plot.fig, slicebox.slice_box, plot_button, detail_plot.fig, \
                           VBox([TS_message, HBox([saveTS_button, TP_number_box, TP_desc_box, TP_saved_dd, delTS_button])])]) 

save_tab_children = VBox([save_instructions, save_all_parameters, save_slices_button, save_feedback])

anplttit = widgets.Label('______________________________________________________________________________________________________________________________________________________________')
plot_tab_children = VBox([anplttit, analysis_poly_order_dd,  analysis_plot.fig,  analysis_plot.fit_statistics, \
                          analysis_plot_zoom_slider.LH_slider, analysis_plot_zoom_slider.RH_slider, slice_trim_button])

strip_chart_children = strip_chart.widget()

filter_children = VBox([filter_label, filter_order, filter_Wn,apply_filter_button])

complete_tabs_children = [slice_tab_children, save_tab_children, plot_tab_children, strip_chart_children, filter_children]
FTAT_tabs = widgets.Tab()
FTAT_tabs.children = complete_tabs_children

for i in range(len(complete_tabs_children)):
    FTAT_tabs.set_title(i, tab_contents[i])
    
FTAT_tabs.layout = Layout(width='100%')

# callbacks

def update_logic(param01):
    selected_tab = FTAT_tabs.selected_index
    #reset the saved files feedback everytime the tabs as changed
    save_feedback.value = ''
    if selected_tab == 0:
        selected_index_changed(None)
    elif selected_tab == 1:
        pass
    elif selected_tab == 2:
        selected_index_changed(None)
    elif selected_tab == 3:
        pass
    else :
        pass

def on_FTAT_tabs_change(change):
    #this is just a wrapper function that will allow me to pass other parameters/objects
    #for now, just call the updater
    update_logic(1)
    
FTAT_tabs.observe(on_FTAT_tabs_change)

HBox([VBox(children = [slicemap.map, time_slices_db_radio], layout=Layout(width='40%')), 
      Box(children = [FTAT_tabs])])